In [ ]:
"""
Ryan Tietjen
Sep 2024
Notebook for use in AWS SageMaker
"""

In [ ]:
import boto3

In [ ]:
import configparser
import torch
config = configparser.ConfigParser()
config.read('config.ini')

device = "cuda" if torch.cuda.is_available() else "cpu"
BATCH_SIZE = int(config["Model Utilization"]["BATCH_SIZE"])
NUM_WORKERS = int(config["Model Utilization"]["NUM_WORKERS"])

print(device)

In [ ]:
from model_creation import create_model
#Create model
model, test_transforms = create_model(config)
model = model.to(device)

In [ ]:
#Load pre exisiting model if applicable
if config["Model Utilization"].getboolean("load_exisiting_model"):
    folder_name = "models"
    model_path = '/'.join((folder_name, config["Model Utilization"]["exisiting_model_name"]))
    model.load_state_dict(torch.load(model_path, weights_only=True))

In [ ]:
import torchvision
#create custom transforms to give a more diverse set of training data
training_transforms = torchvision.transforms.Compose([
    #https://pytorch.org/vision/main/generated/torchvision.transforms.TrivialAugmentWide.html
    torchvision.transforms.TrivialAugmentWide(), #change color, add noise, flip image, etc.
    test_transforms, #original model transforms
])

In [ ]:
import boto3
from torch.utils.data import Dataset
from torchvision import transforms
from PIL import Image
import io

class S3Dataset(Dataset):
    def __init__(self, bucket, prefix, transform=None):
        """
        Args:
            bucket (string): Bucket name.
            prefix (string): Prefix path where images are stored in the bucket.
            transform (callable, optional): Optional transform to be applied on a sample.
        """
        self.s3 = boto3.client('s3')
        self.bucket = bucket
        self.prefix = prefix
        self.transform = transform or transforms.ToTensor()

        # Load the list of files in the bucket
        self.files = []
        self.labels = []
        self.label_map = {}
        paginator = self.s3.get_paginator('list_objects_v2')
        
        for page in paginator.paginate(Bucket=bucket, Prefix=prefix):
            for f in page.get('Contents', []):
                file_path = f['Key']
                if file_path.endswith('.jpg'):  # Filter to include only JPEG images
                    self.files.append(file_path)
                    # Split path and filter out empty strings
                    parts = [part for part in file_path.replace(prefix, '').split('/') if part]
                    if parts and len(parts) > 1:  # Check if there's at least a folder and a file
                        folder_name = parts[0]
                        if folder_name not in self.label_map:
                            self.label_map[folder_name] = len(self.label_map)
                        self.labels.append(self.label_map[folder_name])
                    else:
                        # Handle files directly under prefix or invalid paths by assigning a default label
                        default_label = 'unknown'
                        if default_label not in self.label_map:
                            self.label_map[default_label] = len(self.label_map)
                        self.labels.append(self.label_map[default_label])

    def __len__(self):
        return len(self.files)

    def __getitem__(self, idx):
        key = self.files[idx]
        try:
            obj = self.s3.get_object(Bucket=self.bucket, Key=key)
            image_data = obj['Body'].read()
            #print(f"Fetching image {key} from bucket {self.bucket}")
            #print(f"Size of the data fetched: {len(image_data)} bytes")
            
            if len(image_data) == 0:
                raise ValueError("No data fetched; possibly the file does not exist or is empty.")
            
            img = Image.open(io.BytesIO(image_data))
            img = img.convert("RGB")  # Ensure image is in RGB format
            if self.transform:
                img = self.transform(img)
            label = self.labels[idx]  # This will now be an integer
            return img, label
        except Exception as e:
            print(f"An error occurred while fetching {key}: {str(e)}")
            return None

In [ ]:
from torch.utils.data import DataLoader
from data_setup import get_data_from_S3

# Create an instance of your dataset
s3_bucket = 'food101-images-for-classification'
s3_prefix = 'data/food-101/images'
dataset = S3Dataset(bucket=s3_bucket, prefix=s3_prefix, transform=transforms)

train_data, test_data = get_data_from_S3(dataset,
                 train_ratio=0.8,
                 batch_size=BATCH_SIZE,
                 num_workers=0,
                 train_transform=training_transforms,
                 test_transform=test_transforms)

In [ ]:
print(len(train_data))
print(training_transforms)
print(test_transforms)

In [ ]:
#Setup loss function and optimizer
optimizer = torch.optim.Adam(model.parameters(), 0.001)
loss = torch.nn.CrossEntropyLoss(label_smoothing=0.1)

In [ ]:
#Create SummaryWriter if we want to track results to TensorBoard
if config["Model Utilization"].getboolean("track_results"):
    writer = create_summary_writer(config["Model Utilization"]["experiment_model_name"],
                                    config["Model Utilization"]["experiment_information"])
else:
    writer = None

In [ ]:
!pip install tensorboard

In [ ]:
from model_utilization import train
#Train if applicable
print(config["Model Utilization"]["model_save_name"])
if config["Model Utilization"].getboolean("train"):
    results = train(model=model,
                    train_dataloader=train_data,
                    test_dataloader=test_data,
                    optimizer=optimizer,
                    loss_fn=loss,
                    epochs=int(config["Model Utilization"]["epochs"]),
                    device=device,
                    writer=writer,
                    verbose = config["Model Utilization"].getboolean("verbose"))

In [ ]:
from model_creation import save_model
#Save if applicable
if config["Model Utilization"].getboolean("save_model"):
    model_name = config["Model Utilization"]["model_save_name"]
    print(f"Saving model {model_name}")
    save_model(model, "models", model_name)
